In [30]:
import re
import os
import glob
import zipfile
import requests
from urllib.request import urlretrieve
import json
import pandas as pd

<br>
<br>
<br>
<br>
<br>

## Download the data

In [34]:
# Please don't delete this, just comment it out as needed

%cd /Users/ataciuk/Documents/MDS/525_cloud-computing/

/Users/ataciuk/Documents/MDS/525_cloud-computing


In [46]:
# Necessary metadata
article_id = 14226968  # this is the unique identifier of the article on figshare
url = f"https://api.figshare.com/v2/articles/{article_id}"
headers = {"Content-Type": "application/json"}
output_directory = "data/" # no slash before directory on mac

In [44]:
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)  # this contains all the articles data, feel free to check it out
files = data["files"]             # this is just the data about the files, which is what we want
# files

In [47]:
files_to_dl = ["combined_model_data_parti.parquet.zip"]  ## Please download the partitioned 
for file in files:
    if file["name"] in files_to_dl:
        os.makedirs(output_directory, exist_ok=True)
        urlretrieve(file["download_url"], output_directory + file["name"])

In [48]:
with zipfile.ZipFile(os.path.join(output_directory, "combined_model_data_parti.parquet.zip"), 'r') as f:
    f.extractall(output_directory)

<br>
<br>
<br>
<br>
<br>

## Load the data

In [156]:
df = pd.read_parquet("data/combined_model_data_parti.parquet")

In [157]:
# load the rainfall data, you have to copy it from the Milestone 1 dataset first
observed = pd.read_csv('data/observed_daily_rainfall_SYD.csv')

In [158]:
# removing the data from the index before pivoting as there are weird 12 hr time differences
df['time'] = pd.to_datetime(df['time']).dt.date

In [155]:
df = (df.query("lat_min < -33.86 < lat_max & lon_min < 151.21 < lon_max")  # only Sydney wanted
    .drop(columns=["lat_min", "lat_max", "lon_min", "lon_max"])  # drop geography columns
    .pivot_table(index=['time'], values='rain (mm/day)', columns='model'))  # turn each model to a column
df['observed'] = observed['rain (mm/day)'].values  # add in the observed values
df.columns.name = None  # remove the column cateogry name
print(df.shape)
df


UndefinedVariableError: name 'lat_min' is not defined